# Chatbot Prototype

In [81]:
import json

In [80]:
# Natural Language Processing modules
import nltk
from nltk.stem.lancaster import LancasterStemmer
nltk.download('punkt')
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/machinelearning/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [75]:
# Tensorflow Depedencies
import numpy as np
import tflearn
import tensorflow as tf
import random

## ModelBuilder creates features and trains the neural network

In [85]:
class ModelBuilder(object):
    def __init__(self, seed_data_path):
        with open(seed_data_path) as json_data:
            self.intents = json.load(json_data)
        self.words = []
        self.classes = []
        self.documents = []
        self.ignore_words = ['?']


    def parse_intents_doc(self):     
        # loop through each sentence in our intents patterns
        for intent in self.intents['intents']:
            for pattern in intent['patterns']:
                # tokenize each word in the sentence
                w = nltk.word_tokenize(pattern)
                # add to our words list
                self.words.extend(w)
                # add to documents in our corpus
                self.documents.append((w, intent['tag']))
                # add to our classes list
                if intent['tag'] not in self.classes:
                    self.classes.append(intent['tag'])

        # stem and lower each word and remove duplicates
        self.words = [stemmer.stem(w.lower()) for w in self.words if w not in self.ignore_words]
        self.words = sorted(list(set(self.words)))
        # remove duplicates
        self.classes = sorted(list(set(self.classes)))

    def build_training_data(self):
        # create our training data
        training = []
        output = []
        # create an empty array for our output
        output_empty = [0] * len(self.classes)

        # training set, bag of words for each sentence
        for doc in self.documents:
            # initialize our bag of words
            bag = []
            # list of tokenized words for the pattern
            pattern_words = doc[0]
            # stem each word
            pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
            # create our bag of words array
            for w in self.words:
                if w in pattern_words:
                    bag.append(1)
                else:
                    bag.append(0)

            # output is a '0' for each tag and '1' for current tag
            output_row = list(output_empty)
            output_row[self.classes.index(doc[1])] = 1

            training.append([bag, output_row])

        # shuffle our features and turn into np.array
        random.shuffle(training)
        training = np.array(training)

        # create train and test lists
        train_x = list(training[:,0])
        train_y = list(training[:,1])
        return train_x, train_y

    def train_neural_network(self, train_x, train_y):
        # reset underlying graph data
        tf.reset_default_graph()
        # Build neural network
        net = tflearn.input_data(shape=[None, len(train_x[0])])
        net = tflearn.fully_connected(net, 8)
        net = tflearn.fully_connected(net, 8)
        net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
        net = tflearn.regression(net)

        # Define model and setup tensorboard
        model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
        # Start training (apply gradient descent algorithm)
        model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
        model.save('model.tflearn')

        return model

## ChatBot uses the model and user inputs to return responses

In [90]:
class ChatBot(object):
    ERROR_THRESHOLD = 0.25
    
    def __init__(self, model, words, classes, intents):
        self.model = model
        self.words = words
        self.classes = classes
        self.intents = intents
        
    def clean_up_sentence(self, sentence):
        # tokenize the pattern
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word
        sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
        return sentence_words

    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    def bow(self, sentence, words, show_details=False):
        # tokenize the pattern
        sentence_words = self.clean_up_sentence(sentence)
        # bag of words
        bag = [0] * len(self.words)  
        for s in sentence_words:
            for i, w in enumerate(self.words):
                if w == s: 
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)

        return(np.array(bag))

    def classify(self, sentence):
        # generate probabilities from the model
        results = self.model.predict([self.bow(sentence, self.words)])[0]
        # filter out predictions below a threshold
        results = [[i,r] for i, r in enumerate(results) if r > self.ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append((self.classes[r[0]], r[1]))
        # return tuple of intent and probability
        print return_list
        return return_list

    def response(self, sentence, userID='123', show_details=False):
        print sentence
        results = self.classify(sentence)
        # if we have a classification then find the matching intent tag
        if results:
            # loop as long as there are matches to process
            while results:
                for i in self.intents['intents']:
                    # find a tag matching the first result
                    if i['tag'] == results[0][0]:
                        # a random response from the intent
                        print(random.choice(i['responses']))
                        return

                results.pop(0)

### Train the neural network on the seed data - intents.json

In [86]:
model_builder = ModelBuilder('intents.json')
model_builder.parse_intents_doc()
train_x, train_y = model_builder.build_training_data()
chatbot_model = model_builder.train_neural_network(train_x, train_y)

Training Step: 3999  | total loss: 0.18237 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.18237 - acc: 0.9918 -- iter: 24/25
Training Step: 4000  | total loss: 0.16901 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.16901 - acc: 0.9926 -- iter: 25/25
--
INFO:tensorflow:/Users/machinelearning/Documents/chatbot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


### Fetch class variables for the ChatBot

In [87]:
words = model_builder.words
classes = model_builder.classes
intents = model_builder.intents

In [88]:
chat_bot = ChatBot(chatbot_model, words, classes, intents)

### Test the chatbot here, start asking it questions

In [89]:
chat_bot.response('how do I access CAT')

how do I access CAT
[(u'cat_access', 0.88718957)]
Please visit mightdesk.mightyhive.com/company and click on the CAT button.


In [67]:
chat_bot.response("How do I access DBM")

How do I access DBM
[(u'dbm', 0.96289229)]
Please refer to http://dbm.google.com to login


In [68]:
chat_bot.response("Where can I access DBM?")

Where can I access DBM?
[(u'dbm', 0.95127767)]
Please refer to http://dbm.google.com to login


In [70]:
chat_bot.response("What is CAT?")

What is CAT?
[(u'cat_faq', 0.98465967)]
Cat is a first-party data uploader for targeting your CRM audiences in DBM.


In [74]:
chat_bot.response("What are the recommended creative dimensions?")

What are the recommended creative dimensions?
[(u'creative_dimensions', 0.99409759)]
Please refer to this URL to review accepted creative sizes.
